# LSTM 尼采风格文本生成

TensorFlow 2.0 + Keras + LSTM 序列数据生成。完成**字符级**的尼采风格文本生成任务。

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np

tf.__version__

TensorFlow 2.x selected.


'2.1.0'

## 采样策略

### Softmax Temperature (Softamx 温度)

Softmax Temperature：用于表示采样概率分布的熵，即表示所选择的下一个字符会有多么出人意料或多么可预测。更高的 temperature 得到的是熵更大的采样分布，会生成更加出人意料、更加无结构的生成数据，而更低的温度对应更小的随机性，以及更加可预测的生成数据。

- 更高的温度 = 更随机
- 更低的温度 = 更确定

## 准备数据

先获取语料

In [2]:
path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower() #转换为小写
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 1us/step
Corpus length: 600893


In [3]:
# 提取的字符序列长度
maxlen = 60

# 每 step 个字符采样一个新序列
step = 3

# 保存提取的序列
sentences = []

# 保存目标，下一个字符
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# 语料唯一字符列表
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# 语料唯一字符列表索引
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# 向量化。one-hot 编码为二进制数组
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57


## 模型定义

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars), activation='softmax'))

目标经过 one-hot 编码，所以用 categorical_crossentropy 作为损失。

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 训练语言模型并从中采样

给定一个训练好的模型和一个种子文本片段，可以通过重复以下操作来生成新的文本：

1. 给定目前已生成的文本，总模型中得到下一个字符的概率分布
2. 根据某个温度对分布进行重新加权
3. 根据重新加权后的分布对下一个字符进行随机采样
4. 将新字符添加到文本末尾

采样函数

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

文本生成循环。反复训练并生成文本，没轮过后都使用一系列不同的温度值来生成文本。

In [ ]:
import random
import sys
        
def on_epoch_end(epoch, _):
    """
    函数在每次 epoch 训练后被调用
    """
    print()
    print('----- Generating text after Epoch: %d' % (epoch+1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]: # 使用不同的采样温度生成
        print('----- diversity:', diversity)

        # 随机选择一个种子文本
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400): # 生成 400 个字符
            # 先对目前生成的字符进行 one-hot 编码
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            # 对下一个字符进行采样
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [8]:
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Train on 200278 samples
Epoch 1/60
199680/200278 [============================>.] - ETA: 0s - loss: 1.9693
----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "do penance for
throughout their whole future, if they had a "
do penance for
throughout their whole future, if they had a such the constate of the here the constician and there and self--and there of the persons of such the string the self--and the hard the hard and the heart to the stind the hard to the self-contrist the self-compart of the there and there and a sense of the really and there and there of the readed and the reade soul and there and the hard the self--and there and indelition of the self-say the hard 
----- diversity: 0.5
----- Generating with seed: "do penance for
throughout their whole future, if they had a "
do penance for
throughout their whole future, if they had a so the has a read to makes of the instance, and really the percous of they world of the readous are reading the

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


eased when a relowist, and if, to mexaition? if through soul is ran, to honour.=--he has
at to allee                        
lobqued, favourd--he states spying time, then had a gake
of maskeated by development, by are eve
----- diversity: 1.2
----- Generating with seed: " finer and coarser moralities which have
hitherto prevailed "
 finer and coarser moralities which have
hitherto prevailed herery, whine, hence
thing
of the de, to person may
standinity this brain ephience
entailend to pure similarnerec abouttox
que-terrquethed this realists of all eyk curtesss, necessary" work, and of heart
with devilhwers--you, nungrouss. -all hencl volution-close, but are, in a chuncser.

s    spile of life
--yough!
freedom, shose german,
cases. the, they riseate of mo at sensance, from from larer 
200278/200278 [==============================] - 45s 226us/sample - loss: 1.3151
Epoch 22/60
199552/200278 [============================>.] - ETA: 0s - loss: 1.3131
----- Generating text after Epoch: 22
--